In [50]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import math

In [51]:
##LOADING PLAYER DATA (2015-2016, 2016-2017, 2017-2018, 2018-2019 NBA Seasons)

train_years = [2016,2017,2018]
test_years = [2019]

train_per100 = None
test_per100 = None
train_adv = None
test_adv = None

for year in train_years:
    per100_filepath = '../Data/Player/'+str(year)+'/players_per100.csv'
    adv_filepath = '../Data/Player/'+str(year)+'/players_advanced.csv'
    
    year_per100 = pd.read_csv(per100_filepath)
    year_adv = pd.read_csv(adv_filepath)
    
    #add year to end of team name
    year_per100['Player_year'] = year_per100['Player'] + (['/'+ str(year)] * len(year_per100))
    #add year column
    year_per100['Year'] = year

    #add year to end of team name
    year_adv['Player_year'] = year_adv['Player'] + (['/'+ str(year)] * len(year_adv))
    
    #ADDING ALL_STAR_CALIBER field
    year_adv['All-Star_caliber'] = 0
    year_adv.loc[year_adv.sort_values(by='WS', ascending = False).iloc[:45,:].index,'All-Star_caliber'] = 1
    
    if train_per100 is None:
        train_per100 = year_per100
    else:
        train_per100 = train_per100.append(year_per100, ignore_index = True)
        
    if train_adv is None:
        train_adv = year_adv
    else:
        train_adv = train_adv.append(year_adv, ignore_index = True)
        

        
for year in test_years:
    per100_filepath = '../Data/Player/'+str(year)+'/players_per100.csv'
    adv_filepath = '../Data/Player/'+str(year)+'/players_advanced.csv'
    
    year_per100 = pd.read_csv(per100_filepath)
    year_adv = pd.read_csv(adv_filepath)
    
    #add year to end of team name
    year_per100['Player_year'] = year_per100['Player'] + (['/'+ str(year)] * len(year_per100))
    #add year column
    year_per100['Year'] = year
    
    #add year to end of team name
    year_adv['Player_year'] = year_adv['Player'] + (['/'+ str(year)] * len(year_adv))
    
    #ADDING ALL_STAR_CALIBER field
    year_adv['All-Star_caliber'] = 0
    year_adv.loc[year_adv.sort_values(by='WS', ascending = False).iloc[:45,:].index,'All-Star_caliber'] = 1
    
    if test_per100 is None:
        test_per100 = year_per100
    else:
        test_per100 = test_per100.append(year_per100, ignore_index = True)
        
    if test_adv is None:
        test_adv = year_adv
    else:
        test_adv = train_adv.append(year_adv, ignore_index = True) 

# #drop unnamed and index columns
train_adv = train_adv.drop(['Rk','Unnamed: 19', 'Unnamed: 24'], axis=1)
train_per100 = train_per100.drop(['Rk','Unnamed: 29'], axis=1)
test_adv = test_adv.drop(['Rk','Unnamed: 19', 'Unnamed: 24'], axis=1)
test_per100 = test_per100.drop(['Rk','Unnamed: 29'], axis=1)


In [52]:
##FIX POSITIONS
##TAKEOUT DUPLICATE ROWS FROM PLAYERS THAT CHANGED TEAMS MID-SEASON

##########TRAIN PER100###########

players_w_tot = train_per100.loc[train_per100.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(train_per100)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in train_per100.loc[i,'Pos']:
        #print(p1.loc[i,'Pos_x'])
        train_per100.loc[i,'Pos'] =  train_per100.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if train_per100.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if train_per100.loc[i, 'Tm'] != 'TOT':
            #drop a row
            train_per100 = train_per100.drop(i)

##########TEST PER100###########          
            
players_w_tot = test_per100.loc[test_per100.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(test_per100)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in test_per100.loc[i,'Pos']:
        #print(p1.loc[i,'Pos_x'])
        test_per100.loc[i,'Pos'] =  test_per100.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if test_per100.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if test_per100.loc[i, 'Tm'] != 'TOT':
            #drop a row
            test_per100 = test_per100.drop(i)

##########TRAIN ADV###########

players_w_tot = train_adv.loc[train_adv.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(train_adv)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in train_adv.loc[i,'Pos']:
        train_adv.loc[i,'Pos'] =  train_adv.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if train_adv.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if train_adv.loc[i, 'Tm'] != 'TOT':
            #drop a row
            train_adv = train_adv.drop(i)

##########TEST ADV###########

players_w_tot = test_adv.loc[test_adv.loc[:,'Tm'] == 'TOT','Player']
players_multi  = set(players_w_tot)

for i in range(len(test_adv)):
    #FIX POSITION
    #if positions has '-' take first position
    if '-' in test_adv.loc[i,'Pos']:
        test_adv.loc[i,'Pos'] =  test_adv.loc[i,'Pos'].split('-')[0]

    #ONLY KEEP TOT rows
    if test_adv.loc[i, 'Player'] in players_multi:
        #drop all rows where team is not TOT
        if test_adv.loc[i, 'Tm'] != 'TOT':
            #drop a row
            test_adv = test_adv.drop(i)

In [53]:
##MERGING TABLES

train = train_per100.merge(train_adv, left_on = 'Player_year', right_on = 'Player_year', how = 'left')
test = test_per100.merge(test_adv, left_on = 'Player_year', right_on = 'Player_year', how = 'left')

#Drop duplicate columns from merge
train = train.drop(['Player_y','Pos_y','Age_y','Tm_y','G_y','MP_y'], axis = 1)
test = test.drop(['Player_y','Pos_y','Age_y','Tm_y','G_y','MP_y'], axis = 1)

#Fill NA values with 0
train = train.fillna(0)
test = test.fillna(0)



In [54]:
#add full team/year column

# ADD TEAM ABBREVIATIONS

tm_abrv = np.array([['ATL','Atlanta Hawks'],
['BKN','Brooklyn Nets'],
['BRK','Brooklyn Nets'],
['BOS','Boston Celtics'],
['CHA','Charlotte Hornets'],
['CHI','Chicago Bulls'],
['CLE','Cleveland Cavaliers'],
['DAL','Dallas Mavericks'],
['DEN','Denver Nuggets'],
['DET','Detroit Pistons'],
['GSW','Golden State Warriors'],
['HOU','Houston Rockets'],
['IND','Indiana Pacers'],
['LAC','Los Angeles Clippers'],
['LAL','Los Angeles Lakers'],
['MEM','Memphis Grizzlies'],
['MIA','Miami Heat'],
['MIL','Milwaukee Bucks'],
['MIN','Minnesota Timberwolves'],
['NOP','New Orleans Pelicans'],
['NYK','New York Knicks'],
['OKC','Oklahoma City Thunder'],
['ORL','Orlando Magic'],
['PHI','Philadelphia 76ers'],
['PHX','Phoenix Suns'],
['PHO','Phoenix Suns'],
['POR','Portland Trail Blazers'],
['SAC','Sacramento Kings'],
['SAS','San Antonio Spurs'],
['TOR','Toronto Raptors'],
['UTA','Utah Jazz'],
['WAS','Washington Wizards']])

tm_abrv = pd.DataFrame(tm_abrv, columns = ['Abrv', 'Team_full'])

train_1 = train.merge(tm_abrv, left_on = 'Tm_x', right_on = 'Abrv')
test_1 = test.merge(tm_abrv, left_on = 'Tm_x', right_on = 'Abrv')

train_1['Team_full'] = train_1.apply(lambda row: row.Team_full + '/' + str(row.Year), axis=1)
test_1['Team_full'] = test_1.apply(lambda row: row.Team_full + '/' + str(row.Year), axis=1)



In [55]:
train_1.to_csv('train_players_cleaned.csv')
test_1.to_csv('test_players_cleaned.csv')

In [56]:
test_1

,Player_x,Pos_x,Age_x,Tm_x,G_x,GS,MP_x,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Player_year,Year,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,All-Star_caliber,Abrv,Team_full
0,Alex Abrines\abrinal01,SG,25,OKC,31,2,588,4.4,12.5,0.357,3.3,10.1,0.323,1.2,2.4,0.500,1.0,1.0,0.923,0.4,3.4,3.8,1.6,1.3,0.5,1.1,4.2,13.1,103.0,111,Alex Abrines\abrinal01/2019,2019,6.3,0.507,0.809,0.083,0.9,7.8,4.2,4.3,1.3,0.9,7.9,12.2,0.1,0.6,0.6,0.053,-2.4,-0.9,-3.4,-0.2,0,OKC,Oklahoma City Thunder/2019
1,Steven Adams\adamsst01,C,25,OKC,80,80,2669,8.4,14.1,0.595,0.0,0.0,0.000,8.4,14.1,0.596,2.6,5.1,0.500,6.8,6.5,13.3,2.2,2.0,1.3,2.4,3.6,19.4,120.0,106,Steven Adams\adamsst01/2019,2019,18.5,0.591,0.002,0.361,14.7,14.8,14.7,6.6,2.0,2.4,12.6,16.4,5.1,4.0,9.1,0.163,0.6,2.1,2.7,3.2,1,OKC,Oklahoma City Thunder/2019
2,Deonte Burton\burtode02,SG,25,OKC,32,0,240,6.4,15.9,0.402,1.6,5.3,0.296,4.9,10.7,0.455,1.6,2.3,0.667,0.8,4.7,5.4,1.8,1.2,1.6,1.8,6.0,15.9,94.0,109,Deonte Burton\burtode02/2019,2019,7.0,0.470,0.329,0.146,1.7,10.7,6.0,5.0,1.2,2.8,9.3,16.3,-0.2,0.3,0.1,0.027,-5.6,-0.6,-6.3,-0.3,0,OKC,Oklahoma City Thunder/2019
3,Tyler Davis\davisty01,C,21,OKC,1,0,1,0.0,46.7,0.000,0.0,0.0,0.000,0.0,46.7,0.000,0.0,0.0,0.000,46.7,0.0,46.7,0.0,0.0,0.0,0.0,0.0,0.0,65.0,116,Tyler Davis\davisty01/2019,2019,0.0,0.000,0.000,0.000,100.0,0.0,51.6,0.0,0.0,0.0,0.0,40.7,0.0,0.0,0.0,-0.471,-15.0,-20.5,-35.5,0.0,0,OKC,Oklahoma City Thunder/2019
4,Hamidou Diallo\diallha01,SG,20,OKC,51,3,526,6.7,14.6,0.455,0.4,2.1,0.167,6.3,12.5,0.504,3.2,5.2,0.610,3.4,5.2,8.6,1.5,1.9,0.9,2.0,6.8,16.9,103.0,108,Hamidou Diallo\diallha01/2019,2019,10.1,0.497,0.145,0.358,7.2,12.0,9.5,4.4,1.9,1.6,10.7,16.6,0.1,0.7,0.8,0.069,-4.3,-0.3,-4.7,-0.4,0,OKC,Oklahoma City Thunder/2019
5,Raymond Felton\feltora01,PG,34,OKC,33,0,379,6.8,16.6,0.407,2.5,7.5,0.328,4.3,9.1,0.473,1.5,1.6,0.923,0.5,3.7,4.2,6.4,1.2,0.7,1.7,3.6,17.5,110.0,111,Raymond Felton\feltora01/2019,2019,11.2,0.505,0.452,0.096,1.1,8.4,4.6,18.7,1.2,1.3,9.0,16.6,0.3,0.4,0.7,0.087,-2.3,-1.9,-4.2,-0.2,0,OKC,Oklahoma City Thunder/2019
6,Terrance Ferguson\fergute01,SG,20,OKC,74,74,1931,4.5,10.4,0.429,2.6,7.0,0.366,1.9,3.4,0.560,0.9,1.2,0.725,0.8,2.6,3.4,1.7,1.0,0.4,1.1,5.6,12.4,112.0,112,Terrance Ferguson\fergute01/2019,2019,6.4,0.566,0.673,0.118,1.7,6.0,3.8,4.8,1.0,0.7,9.4,10.6,1.2,1.6,2.8,0.070,-1.5,-0.5,-1.9,0.0,0,OKC,Oklahoma City Thunder/2019
7,Paul George\georgpa01,SF,28,OKC,77,77,2841,11.6,26.5,0.438,4.8,12.4,0.386,6.8,14.1,0.484,7.4,8.9,0.839,1.7,8.6,10.3,5.2,2.8,0.6,3.4,3.5,35.5,116.0,105,Paul George\georgpa01/2019,2019,23.3,0.583,0.469,0.335,3.7,19.6,11.4,17.7,2.8,1.0,10.0,29.5,7.0,4.9,11.9,0.201,4.7,0.7,5.5,5.3,1,OKC,Oklahoma City Thunder/2019
8,Jerami Grant\grantje01,PF,24,OKC,80,77,2612,7.3,14.7,0.497,2.1,5.2,0.392,5.3,9.5,0.555,2.8,3.9,0.710,1.7,5.7,7.5,1.4,1.1,1.8,1.2,3.8,19.5,119.0,108,Jerami Grant\grantje01/2019,2019,13.8,0.592,0.356,0.269,3.7,13.1,8.2,4.2,1.1,3.2,6.8,15.4,4.0,3.3,7.3,0.134,-0.6,0.9,0.3,1.5,1,OKC,Oklahoma City Thunder/2019
9,Donte Grantham\grantdo01,SF,23,OKC,3,0,2,0.0,46.7,0.000,0.0,46.7,0.000,0.0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116,Donte Grantham\grantdo01/2019,2019,-38.1,0.000,1.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.7,0.0,0.0,0.0,-0.946,-25.2,-13.6,-38.9,0.0,0,OKC,Oklahoma City Thunder/2019
